In [38]:
import pandas as pd
import numpy as np

### Reading datasets


In [22]:
df_christies = pd.read_csv('whatmakesdata/christies.csv')
df_sothebys = pd.read_csv('whatmakesdata/SothebysData_clean.csv')

### fixing the year column for the christies dataframe


In [23]:
# Function to fix closeYear column
def fix_year(year):
    if pd.isna(year):  # Handle NaN values
        return None
    year = int(year)  # Convert to integer
    if year > 2000:  # Already in four-digit format
        return year
    elif year >= 50:  # Assume 1950-1999
        return 1900 + year
    else:  # Assume 2000-2049
        return 2000 + year


# Apply function to transform closeYear properly
df_christies["sale_year"] = df_christies["closeYear"].apply(fix_year)

# Drop unnecessary columns
df_christies.drop(columns=["closeYear"], inplace=True)

# Save cleaned dataset
df_christies.to_csv("christies_cleaned.csv", index=False)

# Check result
print(df_christies[["sale_year"]].head())

   sale_year
0       2016
1       2017
2       2013
3       2012
4       2018


### renaming the columns accoridngly and merging the datasets


In [24]:
# Standardize column names
df_sothebys.rename(columns={
    'Title': 'title', 'Artist': 'artist', 'Currency': 'currency',
    'Selling Price': 'price', 'low_estimate': 'low_estimate',
    'high_estimate': 'high_estimate', 'Sold': 'sold',
    'Auction Name': 'auction_title', 'Date': 'sale_year'
}, inplace=True)

df_christies.rename(columns={
    'Art Title': 'title', 'Artist': 'artist', 'Currency': 'currency',
    'price': 'price', 'low_estimate': 'low_estimate',
    'high_estimate': 'high_estimate', 'Sold': 'sold',
    'Auction Title': 'auction_title', 'sale_year': 'sale_year'
}, inplace=True)


# Ensure sale_year is integer
df_sothebys['sale_year'] = pd.to_numeric(
    df_sothebys['sale_year'], errors='coerce')
df_christies['sale_year'] = pd.to_numeric(
    df_christies['sale_year'], errors='coerce')

# Drop unnecessary columns
df_sothebys.drop(columns=['Unnamed: 0', 'Signed',
                 'Category', 'Image'], inplace=True, errors='ignore')
df_christies.drop(columns=['DoC', 'Dimentions', 'Alive?', 'close date',
                  'closeMonth', 'popularity'], inplace=True, errors='ignore')

# Merge datasets
df_merged = pd.concat([df_sothebys, df_christies], ignore_index=True)

# Save merged dataset
df_merged.to_csv('merged_auction_data.csv', index=False)

# Display merged dataset info
df_merged

,artist,title,Location,currency,sale_year,sold,price,low_estimate,high_estimate,auction_title,Birth,Death,Age,Est Accuracy
0,"French School, 18th Century, A white greyhound","Ecole française du XVIIIe siècle, Lévrier blanc",Paris,USD,2022.0,0,0.00,5550.50,7770.70,ANIMALS,NaN,NaN,NaN,NaN
1,François Léon Prieur-Bardin,On the Bosphorus,London,USD,2022.0,1,62830.65,39367.57,52490.10,The Orientalist Sale,NaN,NaN,NaN,NaN
2,Stefano Ussi,A Moroccan Guard,London,USD,2022.0,1,7936.50,5249.01,7873.51,The Orientalist Sale,NaN,NaN,NaN,NaN
3,Ludwig Deutsch,Before the Mosque,London,USD,2022.0,0,0.00,262450.50,393675.75,The Orientalist Sale,NaN,NaN,NaN,NaN
4,Eugène Girardet,The Caravan,London,USD,2022.0,0,0.00,65612.62,91857.67,The Orientalist Sale,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,walter richard sickert,"The Façade of St. Mark's, Venice",NaN,USD,2012.0,1,25000.00,25000.00,37500.00,MODERN BRITISH AND IRISH ART DAY SALE,1860.0,1942.0,82.0,-1.0
2776,robert delaunay,Manège de cochons,NaN,USD,2021.0,1,651000.00,525000.00,840000.00,OEUVRES MODERNES SUR PAPIER / ART IMPRESSIONNI...,1885.0,1941.0,56.0,0.0
2777,henry moore,Two Piece Reclining Figure: Armless,NaN,USD,2018.0,1,985937.50,625000.00,875000.00,IMPRESSIONIST AND MODERN ART EVENING SALE,1898.0,1986.0,88.0,1.0
2778,walter gay,Intérieur Parisien,NaN,USD,2018.0,1,8125.00,4000.00,6000.00,American Art Online,1856.0,1937.0,81.0,1.0


#### removing rows where there is no title and unnecessary columns


In [25]:
df_merged.dropna(subset=['title'], inplace=True)

In [26]:
df_merged.drop(columns=['Location',  'Birth', 'Death',
               'Age', 'Est Accuracy'], inplace=True)  # 'sold',

# Check if all values in the 'currency' column are 'USD'
if df_merged['currency'].nunique() == 1 and df_merged['currency'].iloc[0] == 'USD':
    df_merged.drop(columns=['currency'], inplace=True)

# Save the updated dataframe
df_merged.to_csv('merged_auction_data.csv', index=False)

#### check why some rows have currency value Error


In [27]:
# Add a source column to indicate the origin of each row
df_sothebys['source'] = 'sothebys'
df_christies['source'] = 'christies'

# Merge datasets again with the source column
df_merged_with_source = pd.concat(
    [df_sothebys, df_christies], ignore_index=True)

# Filter rows where currency is not USD
non_usd_rows = df_merged_with_source[df_merged_with_source['currency'] != 'USD']

# Check the source of these rows
print(non_usd_rows['source'].value_counts())
print(len(non_usd_rows['source'].value_counts())*100/len(df_merged))

source
sothebys    97
Name: count, dtype: int64
0.036036036036036036


We see here that all the values with Error in their currency column come from the Sothebys dataset , we have 2 options here, either drop them completely , since we have only 97 such values which represents a mere 0.036% of the whole dataset, or try a different approach to fill them up , since the precentage is too low , we can go ahead and drop these rows


In [28]:
# Drop rows where the currency column has 'Error'
df_merged = df_merged[df_merged['currency'] != 'Error']

# Save the updated dataframe
df_merged.to_csv('merged_auction_data_cleaned.csv', index=False)

# Display the updated dataframe info
df_merged

,artist,title,currency,sale_year,sold,price,low_estimate,high_estimate,auction_title
0,"French School, 18th Century, A white greyhound","Ecole française du XVIIIe siècle, Lévrier blanc",USD,2022.0,0,0.00,5550.50,7770.70,ANIMALS
1,François Léon Prieur-Bardin,On the Bosphorus,USD,2022.0,1,62830.65,39367.57,52490.10,The Orientalist Sale
2,Stefano Ussi,A Moroccan Guard,USD,2022.0,1,7936.50,5249.01,7873.51,The Orientalist Sale
3,Ludwig Deutsch,Before the Mosque,USD,2022.0,0,0.00,262450.50,393675.75,The Orientalist Sale
4,Eugène Girardet,The Caravan,USD,2022.0,0,0.00,65612.62,91857.67,The Orientalist Sale
...,...,...,...,...,...,...,...,...,...
2774,james turrell,Hologram #10,USD,2017.0,1,187500.00,150000.00,250000.00,POST-WAR & CONTEMPORARY ART AFTERNOON SESSION
2775,walter richard sickert,"The Façade of St. Mark's, Venice",USD,2012.0,1,25000.00,25000.00,37500.00,MODERN BRITISH AND IRISH ART DAY SALE
2776,robert delaunay,Manège de cochons,USD,2021.0,1,651000.00,525000.00,840000.00,OEUVRES MODERNES SUR PAPIER / ART IMPRESSIONNI...
2777,henry moore,Two Piece Reclining Figure: Armless,USD,2018.0,1,985937.50,625000.00,875000.00,IMPRESSIONIST AND MODERN ART EVENING SALE


In [29]:
df = pd.read_csv('merged_auction_data_cleaned.csv')

#### Checking why some prices have values of 0


In [30]:
# Filter rows where price is 0
zero_price_rows = df[df['price'] == 0]

# Print the rows
print(zero_price_rows)

# Print the length of these rows
print(f"Number of rows with price 0: {len(zero_price_rows)}")

# Print the percentage of these rows in the whole dataset
percentage_zero_price = (len(zero_price_rows) / len(df)) * 100
print(f"Percentage of rows with price 0: {percentage_zero_price:.2f}%")

                                               artist  \
0     French School, 18th Century, A white greyhound    
3                                      Ludwig Deutsch   
4                                     Eugène Girardet   
13                                    William Simpson   
14                                     Carlos Abascal   
...                                               ...   
1873                             sir eduardo paolozzi   
1912                                  gunther gerzso    
2357                                   lucio fontana    
2550                                  howard hodgkin    
2596                                     andy warhol    

                                                 title currency  sale_year  \
0      Ecole française du XVIIIe siècle, Lévrier blanc      USD     2022.0   
3                                    Before the Mosque      USD     2022.0   
4                                          The Caravan      USD     2022.0   
13 

In [31]:
# Filter rows where price is 0
zero_price_rows = df[df['price'] == 0]

# Count the values in the 'sold' column for these rows
sold_counts = zero_price_rows['sold'].value_counts()

# Print the counts
print(sold_counts)
print(len(df[df['sold'] == 0]))

sold
0    327
Name: count, dtype: int64
327


so we see that the values with 0 in price also happen to have 0 in the value for sold , which means that they're unsold, so we have to drop since we're only interested in those who got sold . Interestingly , all the art works yet to be sold also have 0 in price whcih means that elimanting them in the best option .


In [32]:
# Drop rows where price is 0
df = df[df['price'] != 0]

# Drop the 'sold' column
df.drop(columns=['sold', 'currency'], inplace=True)

# Save the updated dataframe
df.to_csv('final_df.csv', index=False)

# Display the updated dataframe info
df

,artist,title,sale_year,price,low_estimate,high_estimate,auction_title
1,François Léon Prieur-Bardin,On the Bosphorus,2022.0,62830.65,39367.57,52490.10,The Orientalist Sale
2,Stefano Ussi,A Moroccan Guard,2022.0,7936.50,5249.01,7873.51,The Orientalist Sale
5,European School,The Musician,2022.0,4629.63,2624.50,3936.76,The Orientalist Sale
6,Alberto Pasini,The Yesil Mosque in Bursa,2022.0,99206.29,78735.15,104980.20,The Orientalist Sale
7,René Legrand,"Halt by an Inn, North Africa",2022.0,1653.44,2624.50,3936.76,The Orientalist Sale
...,...,...,...,...,...,...,...
2673,james turrell,Hologram #10,2017.0,187500.00,150000.00,250000.00,POST-WAR & CONTEMPORARY ART AFTERNOON SESSION
2674,walter richard sickert,"The Façade of St. Mark's, Venice",2012.0,25000.00,25000.00,37500.00,MODERN BRITISH AND IRISH ART DAY SALE
2675,robert delaunay,Manège de cochons,2021.0,651000.00,525000.00,840000.00,OEUVRES MODERNES SUR PAPIER / ART IMPRESSIONNI...
2676,henry moore,Two Piece Reclining Figure: Armless,2018.0,985937.50,625000.00,875000.00,IMPRESSIONIST AND MODERN ART EVENING SALE


### Reading cleaned merged df


In [33]:
df = pd.read_csv('final_df.csv')
df

,artist,title,sale_year,price,low_estimate,high_estimate,auction_title
0,François Léon Prieur-Bardin,On the Bosphorus,2022.0,62830.65,39367.57,52490.10,The Orientalist Sale
1,Stefano Ussi,A Moroccan Guard,2022.0,7936.50,5249.01,7873.51,The Orientalist Sale
2,European School,The Musician,2022.0,4629.63,2624.50,3936.76,The Orientalist Sale
3,Alberto Pasini,The Yesil Mosque in Bursa,2022.0,99206.29,78735.15,104980.20,The Orientalist Sale
4,René Legrand,"Halt by an Inn, North Africa",2022.0,1653.44,2624.50,3936.76,The Orientalist Sale
...,...,...,...,...,...,...,...
2346,james turrell,Hologram #10,2017.0,187500.00,150000.00,250000.00,POST-WAR & CONTEMPORARY ART AFTERNOON SESSION
2347,walter richard sickert,"The Façade of St. Mark's, Venice",2012.0,25000.00,25000.00,37500.00,MODERN BRITISH AND IRISH ART DAY SALE
2348,robert delaunay,Manège de cochons,2021.0,651000.00,525000.00,840000.00,OEUVRES MODERNES SUR PAPIER / ART IMPRESSIONNI...
2349,henry moore,Two Piece Reclining Figure: Armless,2018.0,985937.50,625000.00,875000.00,IMPRESSIONIST AND MODERN ART EVENING SALE


#### convert sale_year from decimal to normal year


In [34]:
df['sale_year'] = df['sale_year'].astype(int)

#### check for missing values and duplicates


In [35]:
# Check for missing values
df.isnull().sum()

artist           0
title            0
sale_year        0
price            0
low_estimate     0
high_estimate    0
auction_title    0
dtype: int64

In [41]:
df.columns

Index(['artist', 'title', 'sale_year', 'price', 'low_estimate',
       'high_estimate', 'auction_title'],
      dtype='object')

In [36]:
# Check for duplicates
duplicates = df[df.duplicated()]
print(duplicates)
print(f"Number of duplicate rows: {len(duplicates)}")

                                   artist  \
736                        EDWARD ALCOCK    
737           FRANCESCO ZUCCARELLI, R.A.    
738      MANNER OF WILLIAM JONES OF BATH    
739         ENGLISH SCHOOL, 18TH CENTURY    
740   FOLLOWER OF JEAN-BAPTISTE SANTERRE    
...                                   ...   
821            WILLEM CORNELISZ. DUYSTER    
1646                john frederick lewis    
1980                       kazuo shiraga    
1990                        marc chagall    
2068                           paul klee    

                                                  title  sale_year      price  \
736    PORTRAIT OF WALTER KING (D. 1792) OF NAISH HO...       2019    6561.26   
737    DIANA AND CALLISTO, WITH NYMPHS RESTING FROM ...       2019   19683.79   
738    STILL LIFE WITH MIXED FRUIT IN A BASKET AND A...       2019    2460.47   
739    A SPORTING STILL LIFE WITH A GREYHOUND SURROU...       2019    3280.63   
740    PORTRAIT OF A LADY, THREE-QUARTER-LENGTH, IN ..

In [ ]:
from scipy.stats import boxcox
df = df.drop_duplicates()
